In [1]:
import DisorderFromAlphaFold as DISORDER
from Bio.PDB import PDBParser
parser = PDBParser()

pLDDT_THRESHOLD = 70
CONSECUTIVE_DISORDER_THRESHOLD = 20

pdb_path = './TestsAndDemo/test_AF-O60563-F1-model_v4.pdb'

In [2]:
test_structure = parser.get_structure('test_structure', pdb_path)
test_model = test_structure[0]

##### Test functions on a full sequence match

In [3]:
# Investigate pLDDT
pLDDTs = DISORDER.getpLDDTsSubSequenceFromAlphaFoldPDBModel(test_model, 
                                                            startRes=None, 
                                                            endRes=None)
pLDDTs_belowThreshold = pLDDTs[pLDDTs <= pLDDT_THRESHOLD]
print(f'{len(pLDDTs)} residues, {len(pLDDTs_belowThreshold)} of which \'disordered\' (below pLDDT threshold)')

726 residues, 452 of which 'disordered' (below pLDDT threshold)


In [4]:
# Find consecutive pLDDT below threshold
consecutiveDisorderIndices, consecutiveDisorderLengths = DISORDER.getConsecutiveDisorderedFrompLDDTs(pLDDTs, 
                                                                                                     pLDDT_THRESHOLD)
_debug = f'{sum(consecutiveDisorderLengths)} \'disordered\' residues'
_debug += f'spread across {len(consecutiveDisorderLengths)} \'disordered\' stretches'
print(_debug)

452 'disordered' residuesspread across 6 'disordered' stretches


In [5]:
# Filter for lengths above threshold to get a fraction
disorderedFraction, disorderedLengthsFiltered = DISORDER.getDisorderedFractionFrompLDDTs(pLDDTs, 
                                                                                         pLDDT_THRESHOLD, 
                                                                                         CONSECUTIVE_DISORDER_THRESHOLD)
_debug = f'{len(disorderedLengthsFiltered)} stretches above minimum length of {CONSECUTIVE_DISORDER_THRESHOLD} residues'
_debug += f', containing {sum(disorderedLengthsFiltered)} residues overall'
print(_debug)

2 stretches above minimum length of 20 residues, containing 436 residues overall


In [6]:
# Do the same but directly from pdb path
dFrac, dLengths = DISORDER.getDisorderedFractionFromPDB(pdb_path, 
                                                        pLDDT_THRESHOLD, 
                                                        CONSECUTIVE_DISORDER_THRESHOLD)
print(f'This gives a \'disorder\' fraction of {100*dFrac:0.1f}%')

This gives a 'disorder' fraction of 60.1%


##### Now Check Where SubSequencing Is Required

In [7]:
# Take 200 Residues and check lengths
pLDDTs = DISORDER.getpLDDTsSubSequenceFromAlphaFoldPDBModel(test_model, 
                                                            startRes=51, 
                                                            endRes=200)
pLDDTs_belowThreshold = pLDDTs[pLDDTs <= pLDDT_THRESHOLD]
print(f'{len(pLDDTs)} residues, {len(pLDDTs_belowThreshold)} of which \'disordered\' (below pLDDT threshold)')

150 residues, 0 of which 'disordered' (below pLDDT threshold)


##### Now test some sub-sequencing errors

In [8]:
import unittest

class TestSubSequencing(unittest.TestCase):
    _SubSequenceIllogical = lambda: DISORDER.getpLDDTsSubSequenceFromAlphaFoldPDBModel(test_model,
                                                                                   startRes=5,
                                                                                   endRes=1)
    _LargerSubSequence= lambda: DISORDER.getpLDDTsSubSequenceFromAlphaFoldPDBModel(test_model,
                                                                                   startRes=1,
                                                                                   endRes=200000)
    _SubSequenceOutOfBounds= lambda: DISORDER.getpLDDTsSubSequenceFromAlphaFoldPDBModel(test_model,
                                                                                        startRes=200000,
                                                                                        endRes=200005)
    def test_SubSequenceStartGreaterThanFinish(self):
        self.assertRaises(Exception, self._SubSequenceIllogical)
        
    def test_SubSequenceLargerThanAlphaFoldSequence(self):
        self.assertRaises(Exception, self._LargerSubSequence)
        
    def test_SubSequenceOutsideOfAlphaFoldSequence(self):
        self.assertRaises(Exception, self._SubSequenceOutOfBounds)
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_SubSequenceLargerThanAlphaFoldSequence (__main__.TestSubSequencing) ... ok
test_SubSequenceOutsideOfAlphaFoldSequence (__main__.TestSubSequencing) ... ok
test_SubSequenceStartGreaterThanFinish (__main__.TestSubSequencing) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
